<a href="https://colab.research.google.com/github/pranavkalal/Reinforcement-Learning-Model/blob/main/Reinforcement_Learning_Open_Ai_Algorithm_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl

In [ ]:
#LOAD CSV FILE 
from google.colab import files 
  
uploaded = files.upload()
  #importing dataset from the drive
data = pd.read_csv('google_2014_17.csv')



Saving google_2014_17.csv to google_2014_17.csv


In [ ]:
#LOAD CSV FILE 
from google.colab import files 
  
uploaded = files.upload()
  #importing dataset from the drive

data2 = pd.read_csv('google_2018.csv')

Saving google_2018.csv to google_2018.csv


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data2['Date'] = pd.to_datetime(data2['Date'])
data = data.set_index('Date')
data2 = data2.set_index('Date')
print(data.index.min(), data.index.max())
print(data2.index.min(), data2.index.max())
data.head()


2014-03-27 00:00:00 2017-12-29 00:00:00
2018-01-02 00:00:00 2018-08-29 00:00:00


,Unnamed: 0,Open,High,Low,Close,Volume
Date,,,,,,
2014-03-27,0,3.99,4.07,3.87,3.91,24532000
2014-03-28,1,3.93,3.97,3.87,3.88,13189200
2014-03-31,2,3.93,4.02,3.90,4.01,15386000
2014-04-01,3,4.04,4.10,4.02,4.07,18778800
2014-04-02,4,4.12,4.12,4.06,4.06,14839500


In [ ]:
date_split = '2018-01-01'
train = data[:]
test = data2[:]
len(train),len(test)

(949, 167)

In [ ]:
def plot_train_test(train, test, date_split):
    
    data = [
        Candlestick(x=train.index, open=train['Open'], high=train['High'], low=train['Low'], close=train['Close'], name='train'),
        Candlestick(x=test.index, open=test['Open'], high=test['High'], low=test['Low'], close=test['Close'], name='test')
    ]
    layout = {
         'shapes': [
             {'x0': date_split, 'x1': date_split, 'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper', 'line': {'color': 'rgb(0,0,0)', 'width': 1}}
         ],
        'annotations': [
            {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'left', 'text': ' test data'},
            {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'right', 'text': 'train data '}
        ]
    }
    figure = Figure(data=data, layout=layout)
    iplot(figure)

In [ ]:
plot_train_test(train, test, date_split)

In [ ]:
class Environment1:
    
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.profit_percent = []
        self.profit_arr = []
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs
    
    def step(self, act):
        reward = 0
        
        # act = 0: stay, 1: buy, 2: sell
        if act == 1:
            self.positions.append(self.data.iloc[self.t, :]['Close'])
        elif act == 2: # sell
            if len(self.positions) == 0:
                reward = -1
            else:
                profits = 0
                profit_percent = []
                profit_arr = []
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Close'] - p) 
                    profit_arr.append(self.data.iloc[self.t, :]['Close'] - p)
                    profit_percent.append(((self.data.iloc[self.t, :]['Close'] - p) * 100) / p)
                reward += profits
                self.profits += profits
                self.profit_percent = profit_percent
                self.profit_arr = profit_arr
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['Close'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['Close'] - self.data.iloc[(self.t-1), :]['Close'])
        if (self.t==len(self.data)-1):
            self.done=True
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done

In [ ]:
env = Environment1(train)
print(env.reset())
for _ in range(3):
    pact = np.random.randint(3)
    print (pact)
    print(env.step(pact))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.03000000000000025], 0, False)
1
([0.1299999999999999, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.03000000000000025, 0.1299999999999999], 0, False)
1
([0.2500000000000009, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
#def train_dqn(env):
    
class Q_Network(nn.Module):
  def __init__(self,obs_len,hidden_size,actions_n):
            
        super(Q_Network,self).__init__()
            
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
  def forward(self,x):
        h =  self.fc_val(x)
        return (h)

In [ ]:
hidden_size=50
input_size=env.history_t+1
output_size=3
USE_CUDA = False
LR = 0.001

In [ ]:
Q = Q_Network(input_size, hidden_size, output_size)

In [ ]:
Q_ast = copy.deepcopy(Q)

In [ ]:
if USE_CUDA:
    Q = Q.cuda()
loss_function = nn.MSELoss()
optimizer = optim.Adam(list(Q.parameters()), lr=LR)

In [ ]:
epoch_num = 50
step_max = len(env.data)-1
memory_size = 200
batch_size = 50
gamma = 0.97

In [ ]:
obs, reward, done = env.step(5)

In [ ]:
memory = []
total_step = 0
total_rewards = []
total_losses = []
epsilon = 1.0
epsilon_decrease = 1e-3
epsilon_min = 0.1
start_reduce_epsilon = 200
train_freq = 10
update_q_freq = 20
gamma = 0.97
show_log_freq = 5

In [ ]:
start = time.time()
for epoch in range(epoch_num):

    pobs = env.reset()
    step = 0
    done = False
    total_reward = 0
    total_loss = 0

    while not done and step < step_max:

        # select act
        pact = np.random.randint(3)
        if np.random.rand() > epsilon:
            pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
            pact = np.argmax(pact.data)
            pact = pact.numpy()

        # act
        obs, reward, done = env.step(pact)

        # add memory
        memory.append((pobs, pact, reward, obs, done))
        if len(memory) > memory_size:
            memory.pop(0)

        # train or update q
        if len(memory) == memory_size:
            if total_step % train_freq == 0:
                shuffled_memory = np.random.permutation(memory)
                memory_idx = range(len(shuffled_memory))
                for i in memory_idx[::batch_size]:
                    batch = np.array(shuffled_memory[i:i+batch_size])
                    b_pobs = np.array(batch[:, 0].tolist(), dtype=np.float32).reshape(batch_size, -1)
                    b_pact = np.array(batch[:, 1].tolist(), dtype=np.int32)
                    b_reward = np.array(batch[:, 2].tolist(), dtype=np.int32)
                    b_obs = np.array(batch[:, 3].tolist(), dtype=np.float32).reshape(batch_size, -1)
                    b_done = np.array(batch[:, 4].tolist(), dtype=np.bool)

                    q = Q(torch.from_numpy(b_pobs))
                    q_ = Q_ast(torch.from_numpy(b_obs))
                    maxq = np.max(q_.data.numpy(),axis=1)
                    target = copy.deepcopy(q.data)
                    for j in range(batch_size):
                        target[j, b_pact[j]] = b_reward[j]+gamma*maxq[j]*(not b_done[j])
                    Q.zero_grad()
                    loss = loss_function(q, target)
                    total_loss += loss.data.item()
                    loss.backward()
                    optimizer.step()
                    
            if total_step % update_q_freq == 0:
                Q_ast = copy.deepcopy(Q)
                
            # epsilon
            if epsilon > epsilon_min and total_step > start_reduce_epsilon:
                epsilon -= epsilon_decrease

            # next step
            total_reward += reward
            pobs = obs
            step += 1
            total_step += 1

        total_rewards.append(total_reward)
        total_losses.append(total_loss)

        if (epoch+1) % show_log_freq == 0:
            log_reward = sum(total_rewards[((epoch+1)-show_log_freq):])/show_log_freq
            log_loss = sum(total_losses[((epoch+1)-show_log_freq):])/show_log_freq
            elapsed_time = time.time()-start
            print('\t'.join(map(str, [epoch+1, epsilon, total_step, log_reward, log_loss, elapsed_time])))
            start = time.time()
            
#return Q, total_losses, total_rewards

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



Streaming output truncated to the last 5000 lines.
25	0.0999999999999992	23242	94978.8	49744.90586678321	0.008506298065185547
25	0.0999999999999992	23243	94987.8	49746.451967823596	0.010138988494873047
25	0.0999999999999992	23244	94996.8	49747.99806886399	0.008706092834472656
25	0.0999999999999992	23245	95005.8	49749.54416990439	0.007930517196655273
25	0.0999999999999992	23246	95014.8	49751.09027094478	0.007670164108276367
25	0.0999999999999992	23247	95023.8	49752.63637198517	0.007635831832885742
25	0.0999999999999992	23248	95032.8	49754.18247302556	0.008053064346313477
25	0.0999999999999992	23249	95041.8	49755.72857406596	0.00948023796081543
25	0.0999999999999992	23250	95050.8	49757.27467510635	0.008589744567871094
25	0.0999999999999992	23251	95059.8	49758.84245447079	0.02672743797302246
25	0.0999999999999992	23252	95068.6	49760.410233835224	0.010636329650878906
25	0.0999999999999992	23253	95077.4	49761.97801319966	0.008000850677490234
25	0.0999999999999992	23254	95086.2	49763.5457925

In [ ]:
test_env = Environment1(test)
pobs = test_env.reset()
test_acts = []
test_rewards = []

for _ in range(len(test_env.data)-1):
    
    pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
    pact = np.argmax(pact.data)
    test_acts.append(pact.item())
            
    obs, reward, done = test_env.step(pact.numpy())
    test_rewards.append(reward)

    pobs = obs
        
test_profits = test_env.profits
test_profit_percent = test_env.profit_percent
test_profit_arr = test_env.profit_arr

In [ ]:
test_profits

43.80000399999999

In [ ]:
sum(test_profit_percent)/len(test_profit_percent)


11.159356279597253

In [ ]:
test_profit_percent

[14.736842105263166, 7.58187045393134]